In [1]:
from core import database
import isb
import datetime
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

mapbox_access_token = open(".mapbox_token").read()

In [2]:
db = database()

C:\Users\palmarg\AppData\Roaming\Python\Python312\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [29]:
df_stats = db.query("""
select 
    date_trunc('week', d.date) as date,
    case when rt.name in ('Fjölbýlishús', 'Hæð') then 'Fjölbýli' else 'Sérbýli' end as realestate_category_name,
    case when construction_year >= extract(year from d.date) - 5 and (sa.first_sale_date is null or sa.first_sale_date >= d.date) then 'new' else 'old' end as construction_category,
    c.name as city_name,
    avg((case when price > 10e6 and price < 300e6 and r.square_meters > 20 then price / r.square_meters else null end)) as sqm_price,
    count(distinct r.realestate_id) as listings,
    avg((d.date::date - lower(l.span)::date)::int) as listing_days
from listings l
join listing_prices lp
on l.realestate_id = lp.realestate_id
and l.span && lp.span
join realestates r
on lp.realestate_id = r.realestate_id
cross join generate_series(lower(lp.span)::date, (upper(lp.span)-make_interval(days:=1))::date, make_interval(days:=1)) as d(date)
left join (
  select realestate_id, min(registration_date) as first_sale_date
  from sale_agreements
  group by realestate_id         
) sa
on r.realestate_id = sa.realestate_id
join realestate_types rt
on r.realestate_type_id = rt.realestate_type_id
join units u 
on r.unit_id = u.unit_id
join addresses a
on u.address_id = a.address_id
join lands la
on a.land_id = la.land_id
join postals p
on la.postal_id = p.postal_id
join cities c
on p.city_id = c.city_id
join regions reg
on c.region_id = reg.region_id
where rt.name in ('Fjölbýlishús', 'Par/Raðhús', 'Hæð', 'Einbýlishús')
  and reg.name = 'Norðurland eystra'
  and c.name = 'Akureyrarbær'
group by 1,2,3,4
order by 1 desc
""")

In [30]:



df_by_construction_category = df_stats\
.assign(date = lambda r: r.date.dt.date)\
.assign(
    listing_sqm = lambda r: r.sqm_price * r.listings,
    listing_days = lambda r: r.listing_days * r.listings,
)\
.groupby(['date', 'construction_category'], as_index=False)\
.agg(
    listing_sqm = ('listing_sqm', 'sum'),
    listing_days = ('listing_days', 'sum'),
    listings = ('listings', 'sum'),
)\
.assign(
    listing_days = lambda r: r.listing_days / r.listings,
    sqm_price = lambda r: r.listing_sqm / r.listings,
)\
.loc[:, ['date', 'construction_category', 'listing_days', 'sqm_price', 'listings']]

In [31]:
def make_plot(df):

    fig = isb.Figure('core')

    for i, column in enumerate(df.columns):

        fig.add_scatter(
            x = df.index,
            y = df[column],
            line = dict(
                color = fig.colors(i*2)
            ),
            showlegend=False
        )
    
    fig.set_title(
        "Meðaltími <color 0>nýbygginga</color> og <color 2>eldri bygginga</color> á markaði",
    )

    fig.add_annotation(
        x = -0.027,
        y = 1.13,
        showarrow=False,
        font = fig.get_font(size=18),
        text = "<i>Meðalauglýsing nýbygginga á markaði er nú að nálgast 300 daga.<br>Á sama tíma hafa eldri byggingar alls ekki verið eins lengi á sölu, og er meðaltal þeirra núna rétt yfir langtímameðaltali</i>",
        xref='paper', yref='paper',
        xanchor='left',
        align='left'

    )

    fig.update_layout(
        margin_t = 150,
        #title_y = 0.93
    )

    fig.add_logo()


    return fig


df_by_construction_category\
.pivot(index='date', columns='construction_category', values='listing_days')\
.loc[lambda r: r.index >= datetime.date(2019,1,1)]\
.pipe(make_plot)\
.export("time-on-market", scale=1.5)

In [32]:
def make_plot(df):

    fig = isb.Figure('core')

    for i, column in enumerate(df.columns):

        fig.add_scatter(
            x = df.index,
            y = df[column],
            line = dict(
                color = fig.colors(i*2)
            ),
            showlegend=False
        )

        last_row = df.iloc[-1]
        y_row = df.iloc[-52]
        print(y_row)

        fig.add_scatter(
            x = [last_row.name],
            y = [last_row[column]],
            mode = 'markers',
            marker = dict(
                color = fig.colors(i*2),
                size = 8
            ),
            showlegend=False
        )

        fig.add_annotation(
            x = datetime.date(2026,4,1),
            y = last_row[column],
            text = f"{round(last_row[column]/1000,1)} þ.kr.<br>({last_row[column] / y_row[column] - 1:.2%} YoY)",
            font = fig.get_font(size=14),
            showarrow=False,
            bgcolor = fig.bg_color
            
        )
        
        fig.add_scatter(
            x = [last_row.name],
            y = [last_row[column]],
            marker = dict(
                color = fig.bg_color,
                size = 4
            ),
            showlegend=False
        )

    
    fig.set_title(
        "Meðalfermetraverð <color 0>nýbygginga</color> og <color 2>eldri bygginga</color> á markaði",
    )

    fig.add_annotation(
        x = -0.027,
        y = 1.13,
        showarrow=False,
        font = fig.get_font(size=18),
        text = "<i>Auglýst verð nýbygginga hefur staðið nær óhaggað í heilt ár.<br>Á sama tíma hafa eldri byggingar verið að hækka töluvert og hefur verðmunurinn sjaldan verið eins lítill</i>",
        xref='paper', yref='paper',
        xanchor='left',
        align='left'

    )

    fig.update_layout(
        margin_t = 150,
        #title_y = 0.93
    )

    fig.add_logo()


    return fig


df_by_construction_category\
.pivot(index='date', columns='construction_category', values='sqm_price')\
.loc[lambda r: r.index >= datetime.date(2019,1,1)]\
.rolling(4)\
.mean()\
.pipe(make_plot)\
.export("sqmprice-on-market", scale=1.5)

construction_category
new    856564.211330
old    583736.397922
Name: 2024-12-16, dtype: float64
construction_category
new    856564.211330
old    583736.397922
Name: 2024-12-16, dtype: float64


In [35]:
def make_plot(df):

    fig = isb.Figure('core')

    for i, column in enumerate(df.columns):

        fig.add_scatter(
            x = df.index,
            y = df[column],
            line = dict(
                color = fig.colors(i*2)
            ),
            showlegend=False
        )

        last_row = df.iloc[-1]
        y_row = df.iloc[-52]
        print(y_row)

        fig.add_scatter(
            x = [last_row.name],
            y = [last_row[column]],
            mode = 'markers',
            marker = dict(
                color = fig.colors(i*2),
                size = 8
            ),
            showlegend=False
        )

        fig.add_annotation(
            x = datetime.date(2026,5,1),
            y = last_row[column],
            text = f"{int(last_row[column]):,}<br>({last_row[column] / y_row[column] - 1:.2%} YoY)",
            font = fig.get_font(size=14),
            showarrow=False,
            bgcolor = fig.bg_color
            
        )
        
        fig.add_scatter(
            x = [last_row.name],
            y = [last_row[column]],
            marker = dict(
                color = fig.bg_color,
                size = 4
            ),
            showlegend=False
        )

    
    fig.set_title(
        "Fjöldi <color 0>nýbygginga</color> og <color 2>eldri bygginga</color> á markaði",
    )

    fig.add_annotation(
        x = -0.027,
        y = 1.13,
        showarrow=False,
        font = fig.get_font(size=18),
        text = "<i>Nýbyggingum eru þær eignir sem hafa aldrei selst áður, aðrar eignir teljast sem eldri byggingar.</i>",
        xref='paper', yref='paper',
        xanchor='left',
        align='left'

    )

    fig.update_layout(
        margin_t = 150,
        #title_y = 0.93
    )

    fig.add_logo()


    return fig


df_by_construction_category\
.pivot(index='date', columns='construction_category', values='listings')\
.loc[lambda r: r.index >= datetime.date(2019,1,1)]\
.fillna(0)\
.rolling(4)\
.mean()\
.pipe(make_plot)\
.export("total-on-market", scale=1.5)

construction_category
new     71.00
old    164.25
Name: 2024-12-16, dtype: float64
construction_category
new     71.00
old    164.25
Name: 2024-12-16, dtype: float64


In [20]:
def make_plot(df):

    fig = isb.Figure('core')

    for i, column in enumerate(df.columns):

        fig.add_scatter(
            x = df.index,
            y = df[column],
            line = dict(
                color = fig.colors(i*2)
            ),
            showlegend=False
        )

        last_row = df.iloc[-1]
        y_row = df.iloc[-52]
        print(y_row)

        fig.add_scatter(
            x = [last_row.name],
            y = [last_row[column]],
            mode = 'markers',
            marker = dict(
                color = fig.colors(i*2),
                size = 8
            ),
            showlegend=False
        )

        fig.add_annotation(
            x = datetime.date(2026,5,1),
            y = last_row[column],
            text = f"{int(last_row[column]):,}<br>({last_row[column] / y_row[column] - 1:.2%} YoY)",
            font = fig.get_font(size=14),
            showarrow=False,
            bgcolor = fig.bg_color
            
        )
        
        fig.add_scatter(
            x = [last_row.name],
            y = [last_row[column]],
            marker = dict(
                color = fig.bg_color,
                size = 4
            ),
            showlegend=False
        )

    
    fig.set_title(
        "Fjöldi íbúða, sérbýli og fjölbýli, til sölu á hverjum degi",
    )

    fig.add_annotation(
        x = -0.027,
        y = 1.13,
        showarrow=False,
        font = fig.get_font(size=18),
        text = "<i>Talið í fasteignanúmerum til að koma í veg fyrir tvítalningar, þar sem sama eignin getur verið auglýst </i>",
        xref='paper', yref='paper',
        xanchor='left',
        align='left'

    )

    fig.update_layout(
        margin_t = 150,
        #title_y = 0.93
    )

    #fig.add_logo()


    return fig


df_stats\
.assign(date = lambda r: r.date.dt.date)\
.groupby(['date'])\
.agg(listings = ('listings', 'sum'))\
.loc[lambda r: r.index >= datetime.date(2018,1,1)]\
.rolling(4)\
.mean()\
.pipe(make_plot)\
.export("total-on-market--no-pivot", scale=1.5)

listings    2616.5
Name: 2024-12-16, dtype: float64


In [21]:
def make_plot(df):

    fig = isb.Figure('core')
    fig.as_subplot(rows=1, cols=2, shared_yaxes=True)

    df_new = df\
    .loc[lambda r: r.construction_category == 'new']\
    .pivot(index='date', columns='city_name', values='sqm_price')

    df_old = df\
    .loc[lambda r: r.construction_category == 'old']\
    .pivot(index='date', columns='city_name', values='sqm_price')


    base = 700e3

    for i, city_name in enumerate(df_old.columns):

        new_price = df_new.tail(1)[city_name].values[0]
        old_price = df_old.tail(1)[city_name].values[0]

        new_price_12m = df_new.tail(12)[city_name].values[0]
        old_price_12m = df_old.tail(12)[city_name].values[0]

        new_pc = old_price / old_price_12m - 1
        old_pc = new_price / new_price_12m - 1

        fig.add_bar(
            y = [city_name],
            x = [old_price-base],
            text = [f"{old_price/1000:,.0f} þ.kr./m3<br>({old_pc:.2%} YoY)"],
            base=base,
            marker = dict(
                color = fig.colors(i)
            ),
            showlegend = False,
            orientation = 'h',
        )
        fig.add_bar(
            y = [city_name],
            x = [new_price-base],
            text = [f"{new_price/1000:,.0f} þ.kr./m3<br>({new_pc:.2%} YoY)"],
            base=base,
            marker = dict(
                color = fig.colors(i)
            ),
            showlegend = False,
            orientation = 'h',
            row=1, col=2
        )
        
    fig.update_xaxes(
        visible=False
    )

            

    return fig
    


df_stats\
.assign(date = lambda r: r.date.dt.date)\
.loc[lambda r: r.realestate_category_name == 'Fjölbýli']\
.pipe(make_plot)

AttributeError: 'DataFrame' object has no attribute 'construction_category'

In [22]:
def make_plot(df):

    fig = isb.Figure('core')

    df_new = df\
    .loc[lambda r: r.construction_category == 'new']\
    .pivot(index='date', columns='city_name', values='sqm_price')

    df_old = df\
    .loc[lambda r: r.construction_category == 'old']\
    .pivot(index='date', columns='city_name', values='sqm_price')


    for i, city_name in enumerate(df_old.columns):

        new_price = df_new.tail(1)[city_name].values[0]
        old_price = df_old.tail(1)[city_name].values[0]

        fig.add_scatter(
            y = [city_name, city_name],
            x = [old_price, new_price],
            name="New",
            line = dict(
                dash = 'dash',
                width = 3,
                color = fig.colors(i)
            ),
            marker = dict(
                size=14,
                color = fig.colors(i)
            ),
            showlegend = False
        )
        
        fig.add_scatter(
            y = [city_name, city_name],
            x = [old_price, new_price],
            name="New",
            marker = dict(
                size=8,
                color = fig.bg_color,
            ),
            mode = 'markers',
            showlegend = False
        )

            

    return fig
    


df_stats\
.assign(date = lambda r: r.date.dt.date)\
.loc[lambda r: r.realestate_category_name == 'Fjölbýli']\
.pipe(make_plot)

AttributeError: 'DataFrame' object has no attribute 'construction_category'

In [23]:

df_stats\
.assign(date = lambda r: r.date.dt.date)\
.loc[lambda r: r.realestate_category_name == 'Fjölbýli']\
.loc[lambda r: r.construction_category == 'new']\
.pivot(index='date', columns='city_name', values='listing_days')\
.isb.plot()

AttributeError: 'DataFrame' object has no attribute 'construction_category'

In [24]:
df_salary_index = pd.read_csv("https://px.hagstofa.is:443/pxis/sq/a1a47246-889c-4aa1-92e0-b16be1519811", sep=";")\
.rename(columns={"Mánuður": "date", "Vísitölugildi": "salary_index"})\
.assign(date = lambda r: r.date.apply(lambda d: datetime.date(*list(map(int, d.split("M"))), 1)))\
.astype(dict(salary_index = float))\
.astype(dict(date='datetime64[ns]'))\
.set_index('date')

In [25]:
df_property_index = pd.read_csv("data/fasteignavisitala.csv")\
.assign(date = lambda r: r['Earliest Date'].pipe(pd.to_datetime).dt.date)\
.rename(columns={
    "Sameinuð vísitala íbúðaverðs": "value"
})\
.loc[:, ['date', 'value']]\
.set_index('date')



In [26]:
df_stats = db.query("""
select 
    date_trunc('week', d.date) as date,
    case when rt.name in ('Fjölbýlishús', 'Hæð') then 'Fjölbýli' else 'Sérbýli' end as realestate_category_name,
    avg((case when price > 10e6 and price < 300e6 and r.square_meters > 20 then price / r.square_meters else null end)) as sqm_price,
    avg((case when price > 10e6 and price < 300e6 and r.square_meters > 20 then price else null end)) as price,
    count(distinct r.realestate_id) as listings,
    avg((d.date::date - lower(l.span)::date)::int) as listing_days
from listings l
join listing_prices lp
on l.realestate_id = lp.realestate_id
and l.span && lp.span
join realestates r
on lp.realestate_id = r.realestate_id
cross join generate_series(lower(lp.span)::date, (upper(lp.span)-make_interval(days:=1))::date, make_interval(days:=1)) as d(date)
left join (
  select realestate_id, min(registration_date) as first_sale_date
  from sale_agreements
  group by realestate_id         
) sa
on r.realestate_id = sa.realestate_id
join realestate_types rt
on r.realestate_type_id = rt.realestate_type_id
join units u 
on r.unit_id = u.unit_id
join addresses a
on u.address_id = a.address_id
join lands la
on a.land_id = la.land_id
join postals p
on la.postal_id = p.postal_id
join cities c
on p.city_id = c.city_id
join regions reg
on c.region_id = reg.region_id
where rt.name in ('Fjölbýlishús', 'Par/Raðhús', 'Hæð', 'Einbýlishús')
  and reg.name = 'Höfuðborgarsvæðið'
group by 1,2
order by 1 desc
""")

In [15]:
9.396297e+07

93962970.0

In [16]:
def make_plot(df):

    fig = isb.Figure('core')

    fig.add_scatter(
        x = df.index,
        y = df.ratio,
        line = dict(
            color = fig.colors(4)
        ),
        showlegend=False
    )

    fig.set_title(
        "Árslaunamargfeldi meðalíbúðar á höfuðborgarsvæðinu",
    )

    fig.add_annotation(
        x = -0.027,
        y = 1.13,
        showarrow=False,
        font = fig.get_font(size=18),
        text = "<i>Meðalíbúð á höfuðborgarsvæðinu kostar 93.4 m.kr, meðallaun árið 2024 voru 854 þ.kr á mánuði."+
        "<br>Launamargfeldið er hátt, en hefur verið að lækka á síðustu mánuðum.</i>",
        xref='paper', yref='paper',
        xanchor='left',
        align='left'

    )

    fig.add_logo()

    fig.update_layout(
        margin_t = 150,
        #title_y = 0.93
    )
    return fig


df_salary_index\
.join(df_property_index)\
.assign(
    ratio = lambda r: (r.value * 9.396297e+07 / 112.8000) / (r.salary_index * 854e3 * 12 / 1006.7)
)\
.loc[:, ['ratio']]\
.pipe(make_plot)\
.export("cost-to-salary", scale=1.5)